In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
# import nglui.statebuilder as ngstbld

# # this is the EM specific package for querying the EM data
from caveclient import CAVEclient


# from meshparty import trimesh_io, trimesh_vtk
# from meshparty import skeletonize, skeleton_io, skeleton
import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
# from pathlib import Path
# import json

# with open(Path.home() / '.cloudvolume/secrets/'/'cave-secret.json') as f:
#         tokens = json.load(f)
        
# seg_source = 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod'
# cv = cloudvolume.CloudVolume(cloudpath=seg_source, use_https=True, secrets=tokens)

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)

In [ ]:
# if you have not yet setup this computer, uncomment this below line
# paste the token from the website in, and run the line

# client.auth.save_token(token="c14cd7a3e18a1a697716a399afbf5778", overwrite=True)

# then comment or delete the line as you don't need to run it on this computer  again

# Load Premotor table

In [ ]:
pre_to_df = connectome_create.load_pre_to_df(ext='with_class_no_frag')


In [ ]:
pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='matched_typed_with_nt')

# Create an index for pre_to_df with all the levels: cell_class
cell_class, preferred_pool, NT, classification_system, cell_type, segID

In [ ]:
ppmn_idx = pre_to_df.index.to_frame()
ppmn_idx['preferred_pool'] = 'empty'
ppmn_idx['NT'] = 'empty'
ppmn_idx['classification_system'] = 'empty'
ppmn_idx['cell_type'] = 'empty'
ppmn_idx = ppmn_idx[['cell_class', 'preferred_pool', 'NT', 'classification_system', 'cell_type', 'post_pt_root_id']]
pre_to_df.index = pd.MultiIndex.from_frame(ppmn_idx)
pre_to_df


# Reorder the no frags df to have known pmns in the same order along both axes


In [ ]:
All = slice(None)

def reorder_cell_class(pre_to_df,cclss):
    clss_cols = pre_to_df.loc[All,(cclss,All,All,All,All,All,All)].columns.to_frame()
    clss_cols = clss_cols.droplevel(['cell_class','preferred_pool', 'NT', 'classification_system', 'cell_type',])
    clss_cols.index.rename('root_id',inplace=True)

    clss = pre_to_df.loc[(cclss,All,All,All,All,All)]
    clss_idx  = clss.index.to_frame()
    clss.index = clss.index.droplevel(['cell_class','preferred_pool', 'NT', 'classification_system', 'cell_type',])
    clss_idx.index = clss_idx.index.droplevel(['cell_class','preferred_pool', 'NT', 'classification_system', 'cell_type',])

    # clss_cols has the order already figured out in previous code
    # now reduce to the premotor neurons that are still around
    print(clss_cols.shape)
    clss_cols = clss_cols[clss_cols.segID.isin(clss.index.get_level_values('post_pt_root_id'))]
    print(clss_cols.shape)

    clss = clss.loc[clss_cols.segID]

    clss_idx = clss_idx.loc[clss_cols.segID]
    clss_idx = clss_idx.merge(clss_cols,left_index=True,right_on='segID',how='left')

    clss_idx = clss_idx[['cell_class_y','preferred_pool_y', 'NT_y', 'classification_system_y', 'cell_type_y','post_pt_root_id']]
    clss_idx = clss_idx.rename({
            'cell_class_y': 'cell_class',
            'preferred_pool_y': 'preferred_pool',
            'NT_y': 'NT',
            'classification_system_y': 'classification_system',
            'cell_type_y': 'cell_type',
            'post_pt_root_id': 'post_pt_root_id',
        },axis=1)

    # Index has cell_class, ..., post_pt_root_id
    clss.index = pd.MultiIndex.from_frame(clss_idx)

    print('pre_drop:     {}'.format(pre_to_df.shape))
    pre_to_df = pre_to_df.drop(cclss,axis=0,level='cell_class')
    print('post_drop:    {}'.format(pre_to_df.shape))
    pre_to_df = pd.concat([clss,pre_to_df])
    print('post_concat:  {}'.format(pre_to_df.shape))

    print('input order:  {}...'.format(clss_cols.segID.values[0:5]))
    print('output order: {}...'.format(clss.index.get_level_values('post_pt_root_id').values[0:5]))

    print('output clss order: {}'.format(pre_to_df.index.get_level_values('cell_class').unique()))

    return pre_to_df,clss


In [ ]:
# Put the local neurons at the top, pushing the rest down
pre_to_df, local = reorder_cell_class(pre_to_df=pre_to_df,cclss='local')

# Put the intersegmental neurons at the top, pushing the rest down
pre_to_df, intersegmental = reorder_cell_class(pre_to_df=pre_to_df,cclss='intersegmental')

# Put the ascending neurons at the top, pushing the rest down
pre_to_df, ascending = reorder_cell_class(pre_to_df=pre_to_df,cclss='ascending')

# Put the sensory neurons at the top, pushing the rest down
pre_to_df, sensory = reorder_cell_class(pre_to_df=pre_to_df,cclss='sensory')

# Put the descending neurons at the top, pushing the rest down
pre_to_df, descending = reorder_cell_class(pre_to_df=pre_to_df,cclss='descending')


In [ ]:
def reorder_and_pool_motor_neurons(pre_to_df):
    # This one adds the preferred pool to the motor neurons and appends to the pre_to_df at the top

    cclss = 'motor'
    clss_cols = pre_to_mn_df.columns.to_frame()
    clss_cols['preferred_pool'] = 'empty'

    muscle_tuple_dict = utils.get_motor_pool_tuple_dict()
    pool_keys = [
        'thorax_swing',
        'thorax_stance',
        'trochanter_extension',
        'trochanter_flexion',
        'femur_reductor',
        'tibia_extensor',
        'main_tibia_flexor',
        # 'auxiliary_tibia_flexor_A',
        'auxiliary_tibia_flexor_B',
        'auxiliary_tibia_flexor_E',
        'ltm',
        'tarsus_depressor_med_venU',
        'tarsus_depressor_noid',
        ]

    for key in pool_keys:
        mn_tup = muscle_tuple_dict[key]
        clss_cols.loc[mn_tup,'preferred_pool'] = key


    clss_cols = clss_cols.set_index('segID',drop=False)
    clss_cols.index.rename('root_id',inplace=True)
    clss = pre_to_df.loc[(cclss,All,All,All,All,All)]
    clss_idx  = clss.index.to_frame()
    clss.index = clss.index.droplevel(['cell_class','preferred_pool', 'NT', 'classification_system', 'cell_type',])
    clss_idx.index = clss_idx.index.droplevel(['cell_class','preferred_pool', 'NT', 'classification_system', 'cell_type',])

    # clss_cols has the order already figured out in previous code
    # now reduce to the premotor neurons that are still around
    print(clss_cols.shape)
    clss_cols = clss_cols[clss_cols.segID.isin(clss.index.get_level_values('post_pt_root_id'))]
    print(clss_cols.shape)

    clss = clss.loc[clss_cols.segID]

    # reorder the index as well
    clss_idx = clss_idx.loc[clss_cols.segID]
    clss_idx = clss_idx.merge(clss_cols,left_index=True,right_on='segID',how='left')
    clss_idx



    clss_idx = clss_idx[['cell_class','preferred_pool_y', 'NT', 'muscle', 'rank','post_pt_root_id']]
    clss_idx = clss_idx.rename({
            'cell_class': 'cell_class',
            'preferred_pool_y': 'preferred_pool',
            'NT': 'NT',
            'muscle': 'classification_system',
            'rank': 'cell_type',
            'post_pt_root_id': 'post_pt_root_id',
        },axis=1)
    clss_idx['NT'] = 'Glu'


    # Index has cell_class, ..., post_pt_root_id
    clss.index = pd.MultiIndex.from_frame(clss_idx)

    print('pre_drop:     {}'.format(pre_to_df.shape))
    pre_to_df = pre_to_df.drop(cclss,axis=0,level='cell_class')
    print('post_drop:    {}'.format(pre_to_df.shape))
    pre_to_df = pd.concat([clss,pre_to_df])
    print('post_concat:  {}'.format(pre_to_df.shape))

    print('input order:  {}...'.format(clss_cols.segID.values[0:5]))
    print('output order: {}...'.format(clss.index.get_level_values('post_pt_root_id').values[0:5]))

    print('output clss order: {}'.format(pre_to_df.index.get_level_values('cell_class').unique()))

    return pre_to_df,clss

In [ ]:
# Put the descending neurons at the top, pushing the rest down
pre_to_df, motor = reorder_and_pool_motor_neurons(pre_to_df=pre_to_df)

# Review that pre_to_df makes sense

In [ ]:
print(pre_to_df.shape)
print('input: {}'.format(pre_to_df.columns.names))
print('output: {}'.format(pre_to_df.index.names))
print(pre_to_df.columns.get_level_values('cell_class').unique())
print(pre_to_df.index.get_level_values('cell_class').unique())

pre_to_df.head()

In [ ]:
# Save the matrix
connectome_create.save_pre_to_df(pre_to_df,ext='ordered_no_frags')

# First, ensure that looking at the motor neurons gives the same connectome.

In [ ]:
input_tup = (All,All,All,All,All,All)
output_tup = ('motor',All,All,All,All,All)

pre_df_slice = pre_to_df.loc[output_tup,input_tup]

%config InlineBackend.figure_formats = ['png']
%matplotlib inline

cmap = utils.white_dense()

fig = plt.figure(1, figsize = [8,16])
ax = sns.heatmap(np.log10(pre_df_slice.T.to_numpy()+1), cmap=cmap)
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
# cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'log 10 # of synapses', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('All to motor', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

# Not enough here. 